# DataFilters

[![GitHub Workflow Status (main)](https://img.shields.io/github/workflow/status/candoumbe/datafilters/delivery/main?label=main)](https://github.com/candoumbe/DataFilters/actions/workflows/delivery.yml)
[![GitHub Workflow Status (develop)](https://img.shields.io/github/workflow/status/candoumbe/datafilters/integration/develop?label=develop)](https://github.com/candoumbe/DataFilters/actions/workflows/delivery.yml)
[![codecov](https://codecov.io/gh/candoumbe/DataFilters/branch/develop/graph/badge.svg?token=FHSC41A4X3)](https://codecov.io/gh/candoumbe/DataFilters)
[![GitHub raw issues](https://img.shields.io/github/issues-raw/candoumbe/datafilters)](https://github.com/candoumbe/datafilters/issues)
[![Nuget](https://img.shields.io/nuget/vpre/datafilters)](https://nuget.org/packages/datafilters)

A small library that allow to convert a string to a generic [`IFilter`][class-ifilter] object.
Highly inspired by the elastic query syntax, it offers a powerful way to build and query data with a syntax that's not bound to a peculiar datasource.

In [2]:
// Gets the latest version 
#r "nuget:DataFilters" 

using DataFilters;

Installed Packages datafilters, 0.12.0

Let's say we have an API to manage some DC comics data.
The API outputs the following JSON

```json
{
  "id": "vigilante_root",
  "title": "Vigilante",
  "type": "object",
  "properties": {
    "firstname": {
      "required": true,
      "type": "string"
    },
    "lastname": {
      "required": true,
      "type": "string"
    },
    "nickname": {
      "required": true,
      "type": "string"
    },
    "age": {
      "required": true,
      "type": "integer"
    },
    "firstReleasedOn" : {
        "type": "string",
        "format": "date"
    }
    "description": {
      "required": true,
      "type": "string"
    },
    "powers": {
      "required": true,
      "type": "array",
      "items": {
        "type": "string"
      }
    },
    "acolytes": {
      "required": true,
      "type": "array",
      "items": {
        "$ref": "vigilante_root"
      }
    }
  }
}

In [3]:
public class Vigilante
{
    public string Firstname { get; set; }
    public string Lastname { get; set; }
    public string Nickname {get; set; }
    public int Age { get; set; }
    public string Description {get; set;}
    public DateTime? FirstReleasedOn {get; set; }
    public IEnumerable<string> Weapons {get; set; }
    public IEnumerable<string> Powers {get; set;}
    public IEnumerable<Vigilante> Acolytes {get; set;}

    public override string ToString() => $"{Firstname} {Lastname} (A.K.A '{Nickname}')";
};

### Let's build ourself a **Justice** league

In [4]:
#r "nuget:Humanizer"

using System.Text.Json;
using System.Text.Json.Serialization;

Vigilante robin = new () 
{
    Firstname = "Dick",
    Lastname = "Grayson",
    Nickname = "Robin",
    Age = 15,
    Weapons = new [] { "long stick" }
};

Vigilante nightwing = new () 
{
    Firstname = "Dick",
    Lastname = "Grayson",
    Nickname = "Nightwing",
    Age = 28,
    Weapons = new [] { "Tonfa", "Bolas" }
};

Vigilante batman = new () 
{
    Firstname = "Bruce",
    Lastname = "Wayne",
    Nickname = "Batman",
    FirstReleasedOn = On.March.The30th.In(1939),
    Age = 40,
    Weapons = new [] { "batarangs" },
    Acolytes = new []
    {
        robin,
        nightwing
    }
};

Vigilante superman = new () 
{
    Firstname = "Clark",
    Lastname = "Kent",
    Nickname = "Superman",
    Age = 35,
    Powers = new []
    {
        "super strength",
        "heat vison",
        "no shame when wearing a red underwear"
    }
};

Vigilante wonderWoman = new () 
{
    Firstname = "Diana",
    Lastname = "Prince",
    Nickname = "Wonder Woman",
    FirstReleasedOn = In.OctoberOf(1941),
    Powers = new []{ "super strength", "never backdown" }
};

IReadOnlyCollection<Vigilante> justiceLeague = new []{ batman, superman, wonderWoman };
JsonSerializerOptions jsonSerializerOptions = new ()
{
    DefaultIgnoreCondition = JsonIgnoreCondition.WhenWritingDefault,
    WriteIndented = true
};

justiceLeague

Installed Packages Humanizer, 2.14.1

Error: (27,23): error CS0103: Le nom 'On' n'existe pas dans le contexte actuel
(56,23): error CS0103: Le nom 'In' n'existe pas dans le contexte actuel

## The syntax

### Equals

In [5]:
#r "nuget:DataFilters.Expressions, 0.12.0"

using DataFilters.Expressions;

string nicknameIsBatmanQueryString = "Nickname=batman";
IFilter nicknameIsBatmanFilter = nicknameIsBatmanQueryString.ToFilter<Vigilante>();

Console.WriteLine(nicknameIsBatmanQueryString.ToFilter<Vigilante>());
Console.WriteLine($"C# Expression : {nicknameIsBatmanFilter.ToExpression<Vigilante>()}");

Installed Packages DataFilters.Expressions, 0.12.0

{"field":"Nickname","op":"eq","value":"batman"}
C# Expression : item => (item.Nickname == "batman")


### Starts with

In [6]:
using DataFilters.Expressions;

string nicknameStartsWithBatQueryString = "Nickname=bat*";
IFilter nicknameStartsWithBatFilter = nicknameStartsWithBatQueryString.ToFilter<Vigilante>();

Console.WriteLine(nicknameStartsWithBatQueryString.ToFilter<Vigilante>());
Console.WriteLine($"C# Expression : {nicknameStartsWithBatFilter.ToExpression<Vigilante>()}");

{"field":"Nickname","op":"startswith","value":"bat"}
C# Expression : item => item.Nickname.StartsWith("bat")


### Ends with

Search with vigilante which `Nickname` ends with `"man"`

In [7]:
using DataFilters.Expressions;

string nicknameEndsWithManQueryString = "Nickname=*man";
IFilter nicknameEndsWithManFilter = nicknameEndsWithManQueryString.ToFilter<Vigilante>();

Console.WriteLine(nicknameEndsWithManQueryString.ToFilter<Vigilante>());
Console.WriteLine($"C# Expression : {nicknameEndsWithManFilter.ToExpression<Vigilante>()}");

{"field":"Nickname","op":"endswith","value":"man"}
C# Expression : item => item.Nickname.EndsWith("man")


### Contains

Search for vigilante whose `name` contains man

In [8]:
using DataFilters.Expressions;

string nicknameContainsManQueryString = "Nickname=*man*";
IFilter nicknameContainsManFilter = nicknameContainsManQueryString.ToFilter<Vigilante>();

Console.WriteLine(nicknameContainsManQueryString.ToFilter<Vigilante>());
Console.WriteLine($"C# Expression : {nicknameContainsManFilter.ToExpression<Vigilante>()}");

{"field":"Nickname","op":"contains","value":"man"}
C# Expression : item => item.Nickname.Contains("man")


💡 `contains` operator also work on arrays.

In [9]:
using DataFilters.Expressions;

string vigilanteWithSuperStrengthManQueryString = "Powers=*strength*";
IFilter vigilanteWithSuperStrengthManFilter = vigilanteWithSuperStrengthManQueryString.ToFilter<Vigilante>();

Console.WriteLine(vigilanteWithSuperStrengthManQueryString.ToFilter<Vigilante>());
Console.WriteLine($"C# Expression : {vigilanteWithSuperStrengthManFilter.ToExpression<Vigilante>()}");

{"field":"Powers","op":"contains","value":"strength"}
C# Expression : item => item.Powers.Any(Param_0 => Param_0.Contains("strength"))


### Interval expressions

Interval expressions are delimited by an upper and a lower bound syntax.

The generic syntax is `<field> = <min> to <max>` with

- `field` : the name of the property onto which the current interval expression will be applied
- `min` : the lowest bound of the interval
- `max` : the highest bound of the interval

#### Greater than or equal to

In [10]:
using DataFilters.Expressions;

string isOver18QueryString = "Age=[18 TO *[";
IFilter isOver18Filter = isOver18QueryString.ToFilter<Vigilante>();

Console.WriteLine(isOver18QueryString.ToFilter<Vigilante>());
Console.WriteLine($"C# Expression : {isOver18Filter.ToExpression<Vigilante>()}");

{"field":"Age","op":"gte","value":"18"}
C# Expression : item => (item.Age >= 18)


#### Less than or equal to

In [11]:
using DataFilters.Expressions;

string isUnder40QueryString = "Age=]* TO 40]";
IFilter isUnder40Filter = isUnder40QueryString.ToFilter<Vigilante>();

Console.WriteLine(isUnder40QueryString.ToFilter<Vigilante>());
Console.WriteLine($"C# Expression : {isUnder40Filter.ToExpression<Vigilante>()}");

{"field":"Age","op":"lte","value":"40"}
C# Expression : item => (item.Age <= 40)


#### Between

In [12]:
using DataFilters.Expressions;

string isBetween18and40QueryString = "Age=[18 TO 40]";
IFilter isBetween18and40Filter = isBetween18and40QueryString.ToFilter<Vigilante>();

Console.WriteLine(isBetween18and40QueryString.ToFilter<Vigilante>());
Console.WriteLine($"C# Expression : {isBetween18and40Filter.ToExpression<Vigilante>()}");

{"logic":"and","filters":[{"field":"Age","op":"gte","value":"18"},{"field":"Age","op":"lte","value":"40"}]}
C# Expression : Param_0 => ((Param_0.Age >= 18) AndAlso (Param_0.Age <= 40))


💡 You can exclude the lower (respectively upper) bound by using `]` (resp. `[`). 

- `age=]20 TO 35[` means `age` strictly greater than `20` and strictly less than`35`
- `age=[20 TO 35[` means `age` greater than or equal to `20` and strictly less than`35`
- `age=]20 TO 35]` means `age` greater than `20` and less than or equal to `35`

💡 Dates, times and durations must be specified in [ISO 8601 format](https://en.wikipedia.org/wiki/ISO_8601)

Examples :
- `]1998-10-26 TO 2000-12-10[`
- `my/beautiful/api/search?date=]1998-10-26 10:00 TO 1998-10-26 10:00[`
- `]1998-10-12T12:20:00 TO 13:30[` is equivalent to `]1998-10-12T12:20:00 TO 1998-10-12T13:30:00[`

## How to use

### On the client


The client is responsible of building the request with the search criteria by following the syntax describe above

In [13]:
uri = new URL("https://my/wwonderful/api");

uri.setSearch(
{
    "firstname" : "*e*",
    "age": "]18 TO *[",
    "powers": "!\*"
});

console.log(uri);

Error: uri.setSearch is not a function

### On the backend

The backend is responsible of converting query string to [`IFilter`]()

### Key benefits : a predictable syntax predictable

The idea behind `DataFilters` was to provide a syntax that is consistent and easy to integrate with 





[class-multi-filter]: /src/DataFilters/MultiFilter.cs
[class-ifilter]: /src/DataFilters/IFilter.cs
[class-filter]: /src/DataFilters/Filter.cs
[datafilters-expressions]: https://www.nuget.org/packages/DataFilters.Expressions
[datafilters-queries]: https://www.nuget.org/packages/DataFilters.Queries